In [ ]:
from utils.time_series_pvtw import TimeSeriesPVTW
import os

data_source = './data/raw/ACLED_DATA.csv'
output_path =  './data/processed/'
os.makedirs(output_path, exist_ok=True)

output_filepath = f'{output_path}ts_pvtw'

data_cleaner = TimeSeriesPVTW(data_source=data_source)
data_cleaner.save(output_filepath=output_filepath)


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from utils.plot_helper import plot_ts_sequence

from utils.time_series_stl import TimeSeriesSTL
from dateutil.relativedelta import relativedelta

result_path = './results/images/'

data_path = './data/processed/'
df = pd.read_csv(data_path + "ts_pvtw_daily_event.csv") 
ds, target = df.columns
df[ds] = pd.to_datetime(df[ds]) 

start_date = '2020-01-01'
input_seq_length = 56 
output_seq_length = 28 


start_date = pd.to_datetime(start_date).to_pydatetime()
start_date = start_date - relativedelta(days= 5)
end_date = start_date + relativedelta(days= input_seq_length + 2*output_seq_length)
if end_date>df[ds].max():
    end_date = df[ds].max()

df_filtered = df[(df[ds] >= start_date) & (df[ds] <= end_date)].copy()
df_filtered = df_filtered.reset_index(drop=True)

plot_ts_sequence(df_filtered, input_seq_length, output_seq_length, result_path)



In [ ]:
import pandas as pd 
from utils.time_series_stl import TimeSeriesSTL
from utils.plot_helper import plot_pvtw_subplots

data_path = './data/processed/'
result_path = './results/images/filtered/'
gra = 'daily'
os.makedirs(result_path, exist_ok=True)
df_adf = {'Name':[], 'ADF Statistic':[], 'p-value': [], 'Critical Value':[]}

## STL decomposition : starting from 2020 January. 

df = pd.read_csv(data_path + "ts_pvtw_" + gra + ".csv")    

time_series = df.set_index(df.columns[0]) 
time_series.index = pd.to_datetime(time_series.index)  
time_series = time_series.sort_index()

start_date = '2020-01-01' 
time_series_plot = time_series[(time_series.index >=start_date) & (time_series.fatalities<=20) & (time_series.event<=100)]
plot_pvtw_subplots(time_series_plot, gra, result_path)

for feature in df.columns[1:]:
    time_series_analze = time_series[time_series.index >=start_date]
    stl_analyzer = TimeSeriesSTL(time_series_analze, gra, feature, result_path)
    adf_result= stl_analyzer.perform_stl()
    df_adf['ADF Statistic'].append(adf_result[0])
    df_adf['p-value'].append(adf_result[1])
    df_adf['Critical Value'].append(adf_result[4]['5%'])
    df_adf['Name'].append(f"{gra}-{feature}")        

df_adf = pd.DataFrame(df_adf)
df_adf.round(6).to_csv(f'./results/data/adfresult_filtered.csv', index=False)
df_adf.round(6)



In [ ]:
import pandas as pd 
from utils.time_series_stl import TimeSeriesSTL
from utils.plot_helper import plot_pvtw

data_path = './data/processed/'
result_path = './results/images/'
granularity = ['daily', 'monthly']

df_adf = {'Name':[], 'ADF Statistic':[], 'p-value': [], 'Critical Value':[]}

#without filtering the time-frame. 

for gra in granularity:    
    
    df = pd.read_csv(data_path + "ts_pvtw_" + gra + ".csv")    
    time_series = df.set_index(df.columns[0]) 
    time_series.index = pd.to_datetime(time_series.index)  
    time_series = time_series.sort_index()
    
    for feature in df.columns[1:]:
        
        stl_analyzer = TimeSeriesSTL(time_series, gra, feature, result_path)
        adf_result= stl_analyzer.perform_stl()
        df_adf['ADF Statistic'].append(adf_result[0])
        df_adf['p-value'].append(adf_result[1])
        df_adf['Critical Value'].append(adf_result[4]['5%'])
        df_adf['Name'].append(f"{gra}-{feature}")        

   
    start_date = '2020-01-01' #to plot with valid data
    time_series_plot = time_series[time_series.index >=start_date]
    plot_pvtw(time_series_plot, gra, result_path)

df_adf = pd.DataFrame(df_adf)
df_adf.round(6).to_csv(f'./results/data/adfresult.csv', index=False)
df_adf.round(6)
